In [2]:
import pandas as pd
import requests
import pandas as pd
from bs4 import BeautifulSoup
import re
from tqdm import tqdm 
from urllib.parse import urljoin


In [41]:
def data_prep(subset:bool = False) -> pd.DataFrame:
    db_pomoci = pd.read_csv("../db_pomoci.csv")
    df = db_pomoci.rename(columns={"Webová stránka" : "web", "Název":"nazev","E-mail":"email"})
    black_list = ['http://www.dc-brno.cz','www.freeklub.cz', 'http://www.cszs.cz/','http://www.fokustabor.cz/centrum-dusevniho-zdravi-_-komunitni-tym-tabor']
    df = df[(~df["web"].isna()) & (~df['web'].isin(black_list))]
    df['web'] = df['web'].str.replace(r'\s+', '', regex=True)
    df.loc[df['web'].str.contains(r'\.cz/.+'), 'web'] = df['web'].str.replace(r'\.cz/.+', '.cz', regex=True)
    df.loc[df['web'].str.startswith('www'), 'web'] = df['web'].str.replace('^www', 'https://www', regex=True)
    df_agg_tel = df.groupby("web")["Telefon"].agg(list).reset_index()
    df_agg_email = df.groupby("web")["email"].agg(list).reset_index()
    df_agg = pd.merge(df_agg_tel, df_agg_email, on="web")#.drop("index", axis= 1)
    df_agg = df_agg.rename(columns={'web': 'web', 'Telefon': 'Telefon'})
    df_agg = pd.DataFrame(df_agg)
    if subset:
        df_agg = df_agg.head(300) # ubset first 300 rows
    return df_agg

In [17]:
# db_pomoci = pd.read_csv("db_pomoci.csv")

# black_list = ['http://www.dc-brno.cz','www.freeklub.cz, www.cszs.cz','https://www.freeklub.cz']

# df = db_pomoci.rename(columns={"Webová stránka" : "web", "Název":"nazev","E-mail":"email"})

# df = df[(~df["web"].isna()) & (~df['web'].isin(black_list))]

# df.loc[df['web'].str.startswith('www'), 'web'] = df['web'].str.replace('^www', 'https://www', regex=True)

# df_agg_tel = df.groupby("web")["Telefon"].agg(list).reset_index()
# df_agg_email = df.groupby("web")["email"].agg(list).reset_index()
# df_agg = pd.merge(df_agg_tel, df_agg_email, on="web")#.drop("index", axis= 1)
# df_agg = df_agg.rename(columns={'web': 'web', 'Telefon': 'Telefon'})
# df_agg = pd.DataFrame(df_agg)
# df_agg = df_agg[~df_agg['web'].isin(black_list)]
# df_agg.head()
# #df.sort_values("web")


In [18]:
matching_rows = df_agg[df_agg['web'].str.contains('freeklub', case=False, na=False)]

# Display the matching rows
print(matching_rows)

NameError: name 'df_agg' is not defined

In [59]:
def scrape_urls(df):    
    email_regex = re.compile(r"[a-zA-Z0-9._%+-]+@[a-zA-Z0-9.-]+\.[a-zA-Z]{2,}")
    #Phone regex V2
    phone_regex = re.compile(r"""
        (?:
            \+420[-\s]?        # Optional country code +420 followed by an optional space or dash
        )?
        (?:
            \d{3}[-\s]?        # First part of the phone number (3 digits) followed by an optional space or dash
            \d{3}[-\s]?        # Second part of the phone number (3 digits) followed by an optional space or dash
            \d{3}              # Third part of the phone number (3 digits)
            |                  # OR
            \d{3}[-\s]?        # First part of the phone number (3 digits) followed by an optional space or dash
            \d{2}[-\s]?        # Second part of the phone number (2 digits) followed by an optional space or dash
            \d{2}[-\s]?        # Third part of the phone number (2 digits) followed by an optional space or dash
            \d{2}              # Fourth part of the phone number (2 digits)
        )
    """, re.VERBOSE)
    #Phone regex V1
    # phone_regex = re.compile(r"""
    #     (?:
    #         \+420[-\s]?       # Optional country code +420 followed by an optional space or dash
    #     )?
    #     (?:                  # Non-capturing group for the phone number
    #         \d{3}[-\s]?      # First part of the phone number (3 digits)
    #         \d{3}[-\s]?      # Second part of the phone number (3 digits)
    #         \d{3}            # Third part of the phone number (3 digits)
    #         (?:\d{3})?       # Optional fourth part of the phone number (3 digits)
    #     )
    # """, re.VERBOSE)

    # List to collect data
    data = []

    # Function to scrape a single URL
    def scrape_website_main_page(url):
        try:
            response = requests.get(url)
            response.raise_for_status()  # Raises an HTTPError if the response status code is 4XX or 5XX
            soup = BeautifulSoup(response.text, 'html.parser')
            emails = set(email_regex.findall(soup.text))
            phones = set(phone_regex.findall(soup.text))
            return emails, phones
        except requests.RequestException as e:
            #print(f"Error fetching {url}: {e}")
            return set(),set()

    # Function to scrape potential contact pages
    def scrape_website_contacts(base_url, path='', visited_urls=set()):
        full_url = urljoin(base_url, path)
        if full_url in visited_urls:
            return set()  # Return empty set if URL has already been visited
        visited_urls.add(full_url)

        try:
            response = requests.get(full_url)
            response.raise_for_status()
            soup = BeautifulSoup(response.text, 'html.parser')

            emails = set(email_regex.findall(soup.text))
            phones = set(phone_regex.findall(soup.text))

            # Checking for links that may contain contact info
            contact_links = []
            for link in soup.find_all('a', href=True):
                if 'kontakt' in link.text.lower() \
                    or 'contact' in link.text.lower() \
                    or 'kontakty' in link.text.lower() \
                    or 'kdo-jsem' in link.text.lower() \
                    or 'o-nas' in link.text.lower():
                    contact_href = urljoin(full_url, link['href'])
                    if contact_href not in visited_urls:
                        contact_links.append(contact_href)

            # Scrape each potential contact page
            for link in contact_links:
                sub_emails, sub_phones = scrape_website_main_page(link)
                emails.update(sub_emails)
                phones.update(sub_phones)

            return emails, phones
        except requests.RequestException as e:
            #print(f"Error fetching {full_url}: {e}")
            return set(),set()

    # Set of URLs to scrape
    urls = set(df["web"])

    # Loop through the URLs and scrape each one
    visited_urls = set()
    for url in tqdm(urls):
        #print(f"Scraping {url}")
        
        emails, phones = scrape_website_contacts(url, visited_urls=visited_urls)
        ###
        # Condition if main page does not contain contact infromation
        ###    
        # if  ('' in emails) or  ('' in phones):        
        #     emails_2, phones_2 = scrape_website_contacts(url, visited_urls=visited_urls)
        #     emails.add(emails_2)
        #     phones.add(phones_2)
        emails_str = ', '.join(emails)
        phones_str = ', '.join(phones)
        
        # Collect data
        data.append({'Website': url, 'Emails': emails_str, 'Phone Numbers': phones_str})
        #print("Done")
        #print("")
    return data


In [60]:
df = data_prep(subset=True)
#total_rows = df.shape[0]
#df = df.loc[601:total_rows]
df.shape

(300, 3)

In [61]:
df_res = pd.DataFrame(scrape_urls(df))


100%|██████████| 300/300 [08:55<00:00,  1.79s/it]


In [62]:
df_res.to_csv('../data/29_05/df_scraped_v5_300.csv', index=False)

In [112]:
df_scraped_v2_300_600 = pd.read_csv('data/29_05/df_scraped_v3_300.csv')
df_scraped_v2_300_600[(df_scraped_v2_300_600['Emails'].isna()) | (df_scraped_v2_300_600['Phone Numbers'].isna())]

,Website,Emails,Phone Numbers
7,http://www.psychologcb.cz/,NaN,NaN
8,http://www.cheiront.cz/,NaN,NaN
12,http://www.michaelvaclavik.cz/,NaN,+420 608 443 228
13,http://www.cetera.cz,NaN,NaN
15,http://www.kappa-help.cz,NaN,"+420 773 821 003, +420 773 821 001, +420 773 0..."
...,...,...,...
287,http://www.psycholog-tejklova.cz/,NaN,NaN
288,http://www.arcana.cz/,NaN,NaN
289,http://www.psychoterapeut-liberec.cz/,andrea.pirkova@gmail.com,NaN
290,http://psychologtabor.cz/,info@psychologtabor.cz,NaN


In [113]:
df = data_prep(subset=False)
(df.shape[0]-(129 + 121 + 138))/df.shape[0]


0.5519630484988453

In [115]:
df1 = pd.read_csv('data/29_05/df_scraped_v2_300.csv')
df2 = pd.read_csv('data/29_05/df_scraped_v2_300_600.csv')
df3 = pd.read_csv('data/29_05/df_scraped_v2_601_plus.csv')

# Concatenate the DataFrames
concatenated_df = pd.concat([df1, df2, df3], ignore_index=True)

# Save the concatenated DataFrame to a new CSV file
concatenated_df.to_csv('df_scraped_v2.csv', index=False)

(866, 3)

In [94]:
test_emails = ['cheiront@cheiront.cz']
email_regex = re.compile(r"[a-zA-Z0-9._%+-]+@[a-zA-Z0-9.-]+\.[a-zA-Z]{2,}")

for email in test_emails:
    match = email_regex.search(email)
    if match:
        print(f"Matched: {match.group()}")
    else:
        print(f"No match: {email}")

Matched: cheiront@cheiront.cz


# Specific web check

In [ ]:
email_regex = re.compile(r"[a-zA-Z0-9._%+-]+@[a-zA-Z0-9.-]+\.[a-zA-Z]{2,}")
#Phone regex V2
phone_regex = re.compile(r"""
    (?:
        \+420[-\s]?        # Optional country code +420 followed by an optional space or dash
    )?
    (?:
        \d{3}[-\s]?        # First part of the phone number (3 digits) followed by an optional space or dash
        \d{3}[-\s]?        # Second part of the phone number (3 digits) followed by an optional space or dash
        \d{3}              # Third part of the phone number (3 digits)
        |                  # OR
        \d{3}[-\s]?        # First part of the phone number (3 digits) followed by an optional space or dash
        \d{2}[-\s]?        # Second part of the phone number (2 digits) followed by an optional space or dash
        \d{2}[-\s]?        # Third part of the phone number (2 digits) followed by an optional space or dash
        \d{2}              # Fourth part of the phone number (2 digits)
    )
""", re.VERBOSE)
#Phone regex V1
# phone_regex = re.compile(r"""
#     (?:
#         \+420[-\s]?       # Optional country code +420 followed by an optional space or dash
#     )?
#     (?:                  # Non-capturing group for the phone number
#         \d{3}[-\s]?      # First part of the phone number (3 digits)
#         \d{3}[-\s]?      # Second part of the phone number (3 digits)
#         \d{3}            # Third part of the phone number (3 digits)
#         (?:\d{3})?       # Optional fourth part of the phone number (3 digits)
#     )
# """, re.VERBOSE)

# List to collect data
data = []

# Function to scrape a single URL
def scrape_website_main_page(url):
    try:
        response = requests.get(url)
        response.raise_for_status()  # Raises an HTTPError if the response status code is 4XX or 5XX
        soup = BeautifulSoup(response.text, 'html.parser')
        emails = set(email_regex.findall(soup.text))
        phones = set(phone_regex.findall(soup.text))
        return emails, phones
    except requests.RequestException as e:
        #print(f"Error fetching {url}: {e}")
        return set(),set()

# Function to scrape potential contact pages
def scrape_website_contacts(base_url, path='', visited_urls=set()):
    full_url = urljoin(base_url, path)
    if full_url in visited_urls:
        return set()  # Return empty set if URL has already been visited
    visited_urls.add(full_url)

    try:
        response = requests.get(full_url)
        response.raise_for_status()
        soup = BeautifulSoup(response.text, 'html.parser')

        emails = set(email_regex.findall(soup.text))
        phones = set(phone_regex.findall(soup.text))

        # Checking for links that may contain contact info
        contact_links = []
        for link in soup.find_all('a', href=True):
            if 'kontakt' in link.text.lower() \
                or 'contact' in link.text.lower() \
                or 'kontakty' in link.text.lower() \
                or 'kdo-jsem' in link.text.lower() \
                or 'o-nas' in link.text.lower():
                contact_href = urljoin(full_url, link['href'])
                if contact_href not in visited_urls:
                    contact_links.append(contact_href)

        # Scrape each potential contact page
        for link in contact_links:
            sub_emails, sub_phones = scrape_website_main_page(link)
            emails.update(sub_emails)
            phones.update(sub_phones)

        return emails, phones
    except requests.RequestException as e:
        #print(f"Error fetching {full_url}: {e}")
        return set(),set()
web = 'http://www.psychoterapie-iz.cz/index.php/kontakt'
print(scrape_website_main_page(web))

In [ ]:
dfr = pd.read_csv('../data/29_05/df_scraped_v3_300.csv')

dfr[dfr['Phone Numbers'].isna()].shape
# dfr[(dfr['Emails'].isna()) | (dfr['Phone Numbers'].isna())]


In [ ]:
dfr = pd.read_csv('../data/29_05/df_scraped_v5_300.csv')

dfr[dfr['Phone Numbers'].isna()].shape

In [76]:
def sublinks(base_url, path='', visited_urls=set()):
    full_url = urljoin(base_url, path)
    if full_url in visited_urls:
        return set()  # Return empty set if URL has already been visited
    visited_urls.add(full_url)
    
    print(f"Visiting: {full_url}")  # Debug statement

    try:
        response = requests.get(full_url)
        response.raise_for_status()
        soup = BeautifulSoup(response.text, 'html.parser')        

        # Checking for links that may contain contact info
        contact_links = []
        for link in soup.find_all('a', href=True):            
            contact_href = urljoin(full_url, link['href'])
            if contact_href not in visited_urls:
                contact_links.append(contact_href)
        
        print(f"Web {base_url} has {len(contact_links)} subpages")  # Debug statement

        sp = len(contact_links)
        return sp
    except requests.RequestException as e:
        print(f"Error accessing {full_url}: {e}")  # Debug statement
        return 0

In [ ]:
sp = sublinks(url, visited_urls=visited_urls)
sp

In [ ]:
urls = set(df["web"])

# Loop through the URLs and scrape each one
visited_urls = set()
total_pages = 0
total_subpages = 0
for url in urls:
    total_pages +=1
    print(f"Total pages count: {total_pages}")
    
    sp = sublinks(url, visited_urls=visited_urls)
    total_subpages = total_subpages + sp
    print(f"Total subpages count: {total_subpages}")
    print("#####################")